In [3]:
from __future__ import print_function
import pandas as pd 
import numpy as np 
from scipy import stats
#import rpy2
#import rpy2.robjects as robjects
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#from rpy2.robjects import r, pandas2ri
#from rpy2.robjects.packages import importr

#pandas2ri.activate()


#base = importr('base')
#utils = importr('utils')
#ggplot = importr('ggplot2')
#metafor = importr('metafor')

%matplotlib inline 

In [4]:
pd.set_option('display.max_columns', 119)

In [5]:
df_S4=pd.read_excel("../data/Supplement_S3-4_v3.xlsx",header=[0,1],sheet_name="S4")

In [29]:
def q_to_str(i):
    return "%d-%d"%(i.left,i.right)

def clean_str(s):
    return s.replace("℃","C").replace("%","").replace(" ","_").replace("(","_").replace(")","").replace("/","_").replace("__","_")


In [33]:

set_name=["Yield (t /ha)",
          "Plant height (cm)",
          "Root weight (g)",
          "Soil water infiltration rate (mm/min)",
          "Soil water evaporation capacity (mm)",
          "SOM (g/kg)",
          "Olsen-P (mg/kg)"
         ]


def SaveMetaAnalysis(df_in,set_name):
    
    ds_Residual_film=df_in["Soil physical and chemical properties (before test)"]["Residual film (kg/ha)"]
    ds_Residual_film_class=df_in["Soil physical and chemical properties (before test)"]["Residual film class(kg/ha)"]
    ds_Crop_type=df_in["Soil physical and chemical properties (before test)"]["Crop type"]
    ds_qq,bins=pd.qcut(ds_Residual_film,3,retbins=True)
    ds_qq.name="quantile"
    df=pd.merge(df_in[set_name].reset_index(),ds_Residual_film.reset_index(),left_index=True,right_index=True,on="index")
    df=pd.merge(df,ds_qq.reset_index(),left_index=True,right_index=True,on="index")   
    df=pd.merge(df,ds_Crop_type.reset_index(),left_index=True,right_index=True,on="index")   
    df=pd.merge(df,ds_Residual_film_class.reset_index(),left_index=True,right_index=True,on="index")
    
    df["quantile"]=df["quantile"].map(q_to_str)
    
    df.rename(columns={x:clean_str(x) for x in df.columns},inplace=True)
    df.rename(columns={"index":"Reference"},inplace=True)
    set_name=clean_str(set_name)
    
    df["name"]=set_name    
    indx=df["CKmean"].isna().map(lambda x:not x)
    df=df[indx].copy()
    display(df)
    file_name="../data/raw/"+set_name+"_S4.csv"
    df.to_csv(file_name)
    print("File Saved to",file_name)
    

d=interact_manual(SaveMetaAnalysis,df_in=fixed(df_S4),set_name=set_name)

interactive(children=(Dropdown(description='set_name', options=('Yield (t /ha)', 'Plant height (cm)', 'Root we…

In [5]:
df_S3=pd.read_excel("../data/Supplement_S3-4_v3.xlsx",header=[0,1],sheet_name="S3")

In [7]:

def film_thickness_to_str(i):
    return "%.3f-%.3f"%(i.left,i.right)


set_name=["Yield (t/ha)",
          "Temperature (℃)",
          "Soil Water (%)",
         ]


def SaveMetaAnalysis2(df_in,set_name):
        
    df=df_in[set_name].reset_index().copy()
    for dimension in ["Crop classification",
                      "Film type",
                      "Film thickness(mm)",
                      "Mulching cycle",
                      "Mulching method",
                      "Mulching ratio（%)"]:
        ds=df_in["Detailed Information of experiment management"][dimension]
        df=pd.merge(df,ds.reset_index(),left_index=True,right_index=True,on="index")   

        
    ds_ft_q,bins=pd.qcut(df["Film thickness(mm)"],3,retbins=True)
    df["Film thickness class (mm)"]=ds_ft_q.map(film_thickness_to_str)

    ds_mr_q,bins=pd.qcut(df["Mulching ratio（%)"],3,retbins=True,duplicates="drop")
    df["Mulching ratio class (%)"]=ds_mr_q.map(q_to_str)
        
    df.rename(columns={x:clean_str(x) for x in df.columns},inplace=True)
    df.rename(columns={"index":"Reference"},inplace=True)
    set_name=clean_str(set_name)
    
    df["name"]=set_name   
    indx=df["CKmean"].isna().map(lambda x:not x)
    df=df[indx].copy()
    display(df)
    
    file_name="../data/raw/"+set_name+"_S3.csv"
    df.to_csv(file_name)
    print("File Saved to",file_name)
    

d=interact_manual(SaveMetaAnalysis2,df_in=fixed(df_S3),set_name=set_name)

interactive(children=(Dropdown(description='set_name', options=('Yield (t/ha)', 'Temperature (℃)', 'Soil Water…

In [11]:
pd.read_excel("../data/Fig2c-e.xlsx",sheet_name="Farmer_effect").to_csv("../data/raw/Farmer_effect.csv")

In [14]:
pd.read_excel("../data/Fig2c-e.xlsx",sheet_name="Crop type").to_csv("../data/raw/Crop_type.csv")

In [16]:
pd.read_excel("../data/Fig2c-e.xlsx",sheet_name="Mulching time").to_csv("../data/raw/Mulching_time.csv")